#TP Spark
### Noëlie COMTE

##Initialisation de Spark

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 49.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849827 sha256=7f0a3fc54a8634150ec0f62332e66bd87b0b350577529986042536f5a849f819
  Stored in directory: /home/codespace/.cache/pip/wheels/13/92/64/da92a3521323cc629fdf25dd56eb26938e08014c1b57ad3759
Successfully built pyspark

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

# Créer une SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()

# Récupérer le SparkContext
sc = spark.sparkContext


25/01/28 17:41:47 WARN Utils: Your hostname, codespaces-26c329 resolves to a loopback address: 127.0.0.1; using 10.0.0.115 instead (on interface eth0)
25/01/28 17:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 17:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


##TP fait en cours

Crer une liste de nombres, afficher le nombre de partitions

In [3]:

numbers = [1,2,3,4]

#On crée la RDD

rdd = sc.parallelize(numbers)

#On affiche le RDD

rdd.collect()

[1, 2, 3, 4]

In [4]:
rdd.getNumPartitions() #nombre de partitions

1

25/01/28 17:42:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Créer la liste

- Paris, 75
- Nice, 06
- Marseille, 13

Créer un rdd_city a partir de cette liste puis afficher les villes en majuscule

In [5]:
city = [("Paris", 75),("Nice", 6),("Marseille", 13)]

rdd_city = sc.parallelize(city) #transformation n°1

rdd_upper = rdd_city.map(lambda x: (x[0].upper, x[1])) #transformation n°2 #On enlève vonlontairement les parenthèses pour tester les erreurs puis on les remets

rdd_upper.collect() #action

[(<function str.upper()>, 75),
 (<function str.upper()>, 6),
 (<function str.upper()>, 13)]

*Sans collect il n'y a pas d'erreur, avec collect il y a une erreur.*

*Pendant la transformation, il n'affiche pas l'erreur, il ne va l'afficher que pendant l'action.*

*Spark est lazy.*

*Il execute le DAG pendant l'action car il optimise*

In [6]:
rdd_upper = rdd_city.map(lambda x: (x[0].upper(), x[1])) #transformation n°2 #On a corrigé l'erreur

rdd_upper.collect() #action

[('PARIS', 75), ('NICE', 6), ('MARSEILLE', 13)]

Afficher les 5 premières lignes du fichier Hamlet

In [7]:
rdd = sc.textFile('hamlet.txt')
print(rdd.take(5))

['Hamlet by William Shakespeare', 'Characters in the Play', '======================', 'THE GHOST', 'HAMLET, Prince of Denmark, son of the late King Hamlet and Queen Gertrude']


*L'utilisation de take à la place de collect permet d'afficher un nombre précis de lignes à afficher*

Splitter les lignes en utilisant l'espace comme délimiteur pour n'avoir que les mots du texte

On fait appel à flatMap

In [8]:
rdd_mots = rdd.flatMap(lambda line: line.split(" "))
rdd_mots.take(20)

['Hamlet',
 'by',
 'William',
 'Shakespeare',
 'Characters',
 'in',
 'the',
 'Play',
 '======================',
 'THE',
 'GHOST',
 'HAMLET,',
 'Prince',
 'of',
 'Denmark,',
 'son',
 'of',
 'the',
 'late',
 'King']

Compter combien de fois apparaissent les mots en faisant des couples

In [9]:
rdd_couple = rdd_mots.map(lambda mots: (mots,1))
rdd_count = rdd_couple.reduceByKey(lambda x,y: x+y)
rdd_count.take(5)

[('Hamlet', 8),
 ('by', 13),
 ('William', 1),
 ('Shakespeare', 1),
 ('Characters', 1)]

In [10]:
rdd_count.explain()

AttributeError: 'PipelinedRDD' object has no attribute 'explain'

*Explain ne fonctionne pas sur les RDD mais seulement sur les dataframe*

Convertir en dataframe et on affiche

In [11]:
df1 = rdd_count.toDF()
df1.printSchema()


root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [12]:
df1.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|              Hamlet|  8|
|                  by| 13|
|             William|  1|
|         Shakespeare|  1|
|          Characters|  1|
|                  in| 41|
|                 the|101|
|                Play|  1|
|=================...|  1|
|                 THE|  1|
|               GHOST|  1|
|             HAMLET,|  2|
|              Prince|  2|
|                  of| 80|
|            Denmark,|  2|
|                 son|  3|
|                late|  3|
|                King|  9|
|                 and| 82|
|               Queen|  1|
+--------------------+---+
only showing top 20 rows



*.collect sur le rdd*

*.show sur le dataframe*

Faire un count sur hamlet_gros

In [13]:
rdd_gros = sc.textFile('hamlet_gros.txt')
rdd_gros.count()

10744

Faire un cache

--> prendre rdd et faire .persist()

In [14]:
rdd_gros.persist()
rdd_gros.count()

10744

Faire un autre count

*Le but c'est de déclencher la transformation persist avec l'action count*

*La première fois, c'est plus lent car le persist n'est pas encore déclenché*


In [15]:
rdd_gros.count()

10744